In [ ]:
import holoviews as hv, param, pandas as pd
import panel as pn
import numpy as np
import bokeh

In [ ]:
from colorcet import cm
import colorcet
import datashader as ds
from holoviews.operation.datashader import rasterize, shade, spread
from holoviews.element.tiles import StamenTerrain
from holoviews.util.transform import lon_lat_to_easting_northing
from datashader.utils import lnglat_to_meters
from holoviews.element import tiles as hvts
from collections import OrderedDict as odict

In [ ]:
hv.extension('bokeh', logo=False)

In [ ]:
class NYCTaxiExplorer(param.Parameterized):
    usecols = ['dropoff_x','dropoff_y','pickup_x','pickup_y','dropoff_hour','pickup_hour','passenger_count']
    df=pd.read_csv("taxi2016.csv")
    koordinatDonusum = lon_lat_to_easting_northing(df["dropoff_longitude"],df["dropoff_latitude"])
    df["dropoff_x"], df["dropoff_y"] = koordinatDonusum[0], koordinatDonusum[1] 
    koordinatDonusum = lon_lat_to_easting_northing(df["pickup_longitude"],df["pickup_latitude"])
    df["pickup_x"], df["pickup_y"] = koordinatDonusum[0], koordinatDonusum[1] 

    ##df.rename(columns={"dropoff_longitude":"dropoff_x","dropoff_latitude":"dropoff_y","pickup_longitude":"pickup_x","pickup_latitude":"pickup_y"},inplace=True)
    df["pickup_hour"] = pd.to_datetime(np.array(df["tpep_pickup_datetime"].values,dtype=np.datetime64)).hour.tolist()
    df["dropoff_hour"] = pd.to_datetime(np.array(df["tpep_dropoff_datetime"].values,dtype=np.datetime64)).hour.tolist()
    df = df[usecols]
    fields= odict([("passenger_count","passenger_count")])
    opts = dict(width=1000,height=600,xaxis=None,yaxis=None,bgcolor='black',show_grid=False)
    cmaps = ['fire','bgy','bgyw','bmy','gray','kbc']
    maps   = ['EsriImagery', 'EsriUSATopo', 'EsriTerrain', 'CartoMidnight', 'StamenWatercolor', 'StamenTonerBackground']
    bases  = odict([(name, getattr(hvts, name)().relabel(name)) for name in maps])
    gopts  = hv.opts.Tiles(responsive=True, xaxis=None, yaxis=None, bgcolor='black', show_grid=False)
    aggfns = odict([(f.capitalize(),getattr(ds,f)) for f in ['count','sum','min','max','mean','var','std']])
    norms  = odict(Histogram_Equalization='eq_hist', Linear='linear', Log='log', Cube_root='cbrt')
    alpha      = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    cmap       = param.ObjectSelector(colorcet.palette['fire'], objects={c:colorcet.palette[c] for c in cmaps})
    hour       = param.Range(default=(0, 24), bounds=(0, 24))
    location   = param.ObjectSelector(default='dropoff', objects=['dropoff', 'pickup'])
    basemap    = param.Selector(bases)
    spreading  = param.Integer(0, bounds=(0, 5))
    agg_fn     = param.Selector(aggfns)
    field      = param.Selector(fields)
    normalization = param.Selector(norms)
    data_opacity  = param.Magnitude(1.00)

In [ ]:
    @param.depends('alpha', 'basemap')
    def tiles(self):
        return self.basemap.opts(self.gopts).opts(alpha=self.alpha)
    @param.depends('field', 'agg_fn')
    def aggregator(self):
        field = None if self.field == "counts" else self.field
        return self.agg_fn(field)
    @param.depends('location','hour')
    def points(self):
        points = hv.Points(self.df, kdims=[self.location+'_x', self.location+'_y'], vdims=['dropoff_hour','passenger_count'])
        if self.hour != (0, 24): points = points.select(dropoff_hour=self.hour)
        return points
    def viewable(self,**kwargs):
        rasterized = rasterize(hv.DynamicMap(self.points), aggregator=self.aggregator, width=800, height=400)
        shaded     = shade(rasterized, cmap=self.param.cmap, normalization=self.param.normalization)
        spreaded   = spread(shaded, px=self.param.spreading, how="add")
        dataplot   = spreaded.apply.opts(alpha=self.param.data_opacity, show_legend=False)
        
        return hv.DynamicMap(self.tiles) * dataplot 

axi = NYCTaxiExplorer(name="NYC Taxi Trips")<br>
=pn.Row(taxi.param, taxi.viewable())<br>
.servable()

In [ ]:
def run():
    taxi = NYCTaxiExplorer(name="NYC Taxi Trips")
    s=pn.Row(taxi.param, taxi.viewable())
    return s

In [ ]:
pn.serve(run,port=3002)